In [1]:
import os
import sys
import math

import subprocess
import altair as alt
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import pyspark.sql.functions as F
from prompt_toolkit.styles.style import default_priority
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = (
    SparkSession.builder
    .appName('Airport Traffic')
    .master('local[4]')
    .config('spark.executor.memory', '2g')
    .config('spark.executor.cores', '2')
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "1")
    .config("spark.dynamicAllocation.maxExecutors", "4")
    .config('spark.executor.memoryOverhead', '512m')
    .config("spark.driver.memory", "2g")
    .config("spark.driver.maxResultSize", "2g")
    .config('spark.sql.adaptive.enabled', 'true')
    .config('spark.sql.adaptive.coalescePartitions.enabled', 'true')
    .config('spark.sql.adaptive.advisoryPartitionSizeInBytes', '64mb')
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config('spark.dynamicAllocation.executorIdleTimeout', '60s')
    .config('spark.sql.autoBroadcastJoinThreshold', '512mb')
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/16 14:17:36 WARN Utils: Your hostname, Zygimantass-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.43.73.162 instead (on interface en0)
26/01/16 14:17:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/16 14:17:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_path_songs = r'/Users/zygimantas/Documents/DataSets/Songs/songs.csv'
df_path_artists = r'/Users/zygimantas/Documents/DataSets/Songs/artists.csv'

In [5]:
spark = SparkSession.builder.appName("CSVToParquet").getOrCreate()

26/01/16 14:17:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
df_songs_csv = spark.read.csv(df_path_songs, header=True, inferSchema=True)

In [8]:
df_songs_csv.write.parquet(r'/Users/zygimantas/Documents/DataSets/Songs/songs.parquet')

In [9]:
df_artists_csv = spark.read.csv(df_path_artists, header=True, inferSchema=True)

In [10]:
df_artists_csv.write.parquet(r'/Users/zygimantas/Documents/DataSets/Songs/artists.parquet')

In [11]:
df_artist_path = r'/Users/zygimantas/Documents/DataSets/Songs/artists.parquet'
df_songs_path = r'/Users/zygimantas/Documents/DataSets/Songs/songs.parquet'

In [12]:
df_artists = spark.read.parquet(df_artist_path, header=True, inferSchema=True)

In [13]:
df_songs = spark.read.parquet(df_songs_path, header=True, inferSchema=True)